We aim to compute the flow: 

$$\phi(x, g \cdot x) = \sum\limits_{n \geq 0} \sum\limits_{0 \leq m \leq 2^n} \sum\limits_{z \in (-mg-R_{2^n}) \cdot x} \frac{1_A(z) - 1_B(z)}{|R_{2^{n+1}}|} $$

A few useful facts: 
  - $|R_{2^{n+1}}| = 2^{n+1} \times 2^{n+1} = 2^{2n+2}$ 
  - $ g \in \{(1,0), (0,1), (1,1)\}$

In [1]:
###############################
import math
import time
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
###############################

In [2]:
###############################
# Global variables
###############################
c = (0.5, 0.5)                       # coordinate for center of shapes
r = 0.49                             # radius for circle
s = math.sqrt(math.pi*r**2)          # side length for square
#g = (1,0)                            # direction, choose out of S = {(1,0), (0,1), (1,1)}
u1 = (1/math.sqrt(2),1/math.sqrt(3)) # vector 1
u2 = (1/math.sqrt(4),1/math.sqrt(5)) # vector 2

def checkValid():
    if (r > 0.5 or s > 1):
        raise Exception("Shapes are too big!")


In [3]:
###############################
# Characteristic functions
###############################

# Characteristic function of circle
# Input: a point (tuple) z in the 2D torus.
# Output: 1 if z in the circle (including boundary), 0 otherwise
def oneA(z):  
    if math.sqrt((z[0]-c[0])**2 + (z[1]-c[1])**2) <= r:
        return 1
    else:
        return 0

# Characterizatic function of square
# Input: a point (tuple) z in the 2D torus.
# Output: 1 if z in the square (including boundary), 0 otherwise
def oneB(z):
    if abs(z[0]-c[0]) <= s/2 and abs(z[1]-c[1]) <= s/2:
        return 1
    else:
        return 0


In [4]:
###############################
# Calculate innermost summation
###############################

# Generate set -mg-R_{2^n} \cdot x: {n1*u1 + n2*u2 + x: (n1,n2) \in -mg-R_{2^n}}, 
#  where -mg-R_{2^n} = {(a,b): (a,b)=-m*g-d for some d in R_{2^n}}
# Input: u1, u2 tuples representing vectors; N upper bound on i,j
# Output: list of points in torus satisfying {i*u1 + j*u2: 0 <= i,j < N}

def make_set_z(n, m, g, x, show = False):
    F = list()
    neg_mg = tuple([-m*i for i in g])
    for i in range(2**n):
        for j in range(2**n):
            n1, n2 = tuple(map(lambda k, l: k - l, neg_mg, (i,j)))
            x_coord = (n1*u1[0]+n2*u2[0]+x[0])%1
            y_coord = (n1*u1[1]+n2*u2[1]+x[1])%1
            F.append((x_coord, y_coord))
            
            if show:
                print("n1 = " + str(n1))
                print("n2 = " + str(n2))
                print("coordinate is: ")
                print((x_coord, y_coord))
            
    return F



def sum3(n, m, g, x, show = False):
    sum = 0
    if show:
        Z = make_set_z(n,m,g,x, show = True)
    else:
        Z = make_set_z(n,m,g,x)
    for z in Z:
        sum += oneA(z) - oneB(z)
        if show:
            print("adding: " + str(oneA(z) - oneB(z)))
    return sum/(2**(2*n+2))



In [5]:
###############################
# Calculate middle summation
###############################

def sum2(n, g, x):
    sum = 0
    for m in range(2**n):
        sum += sum3(n,m,g,x)
    return sum


In [6]:
#####################################################
# Calculate outermost summation (aka the flow)
#####################################################

def phi(x, g):
    sum = 0
    #print("iteration " + str(x))
    for n in range(6):
        sum += sum2(n,g,x)
    return sum


In [7]:
###############################
# Graph the flow
###############################

# I would love to use the code from:
# https://jakevdp.github.io/PythonDataScienceHandbook/04.12-three-dimensional-plotting.html 
# But alas this uses numpy, and I (maybe foolishly) coded everything above without numpy

# https://likegeeks.com/3d-plotting-in-python/

def graph(g, show=False):
    GR = 21 # granularity of graph
    
    x = np.linspace(0, 1, GR)[:-1] # don't include x=1
    y = np.linspace(0, 1, GR)[:-1]
    
    tic = time.perf_counter()
    z = np.array([phi((i.item(), j.item()), g) for j in y for i in x])
    Z = z.reshape(len(y), len(x))
    toc = time.perf_counter()
    if show: print("time elapsed: " + str(toc-tic) + " sec")
    
    plt.pcolor(Z, cmap="Blues")
    plt.colorbar()
    plt.show()
    

In [8]:
####################################
# Check flow 
####################################

# Calculate 1A(x)-1B(x)+phi(x) for a given point x. Value whould be approx. 0.
# Input: point x
# Output: 1A(x)-1B(x)+phi(x)

def confirm(x):
    outflow = phi(x,(1,0)) + phi(x,(0,1)) + phi(x,(1,1))
    x1 = ((-1*u1[0]+0*u2[0]+x[0])%1, (-1*u1[1]+0*u2[1]+x[1])%1) # (-1,0) \cdot x
    x2 = ((0*u1[0]-1*u2[0]+x[0])%1, (0*u1[1]-1*u2[1]+x[1])%1)   # (0,-1) \cdot x
    x3 = ((-1*u1[0]-1*u2[0]+x[0])%1, (-1*u1[1]-1*u2[1]+x[1])%1) # (-1,-1) \cdot x
    inflow = phi(x1,(1,0)) + phi(x2,(0,1)) + phi(x3,(1,1))
    return oneA(x) - oneB(x) - outflow + inflow


# Run confirm(x) for many points in the torus
# Input: N/A
# Output: n by n array of 1A(x)-1B(x)+phi(x) values

def run_confirm():
    GR = 11                          #our n value
    x = np.linspace(0, 1, GR)[:-1] 
    y = np.linspace(0, 1, GR)[:-1]
    z = np.array([confirm((i.item(), j.item())) for j in y for i in x])
    Z = z.reshape(len(y), len(x))
    return Z
    

In [9]:
####################################
# main
####################################

# Testing functions.

def main():
    checkValid()
    
    
    g = (1,1)
    x = (0.5,0.5)

    #graph(g, show=True)
    
    print(run_confirm()) # seems to (almost) all be within 0.01 of 0
    

if __name__ == "__main__":
    main()

[[ 0.00170898  0.00952148 -0.00170898 -0.00366211 -0.00244141  0.0078125
   0.0065918  -0.0065918  -0.00097656  0.00097656]
 [ 0.00341797  0.01098633 -0.00366211 -0.00415039 -0.00146484  0.00561523
   0.00610352 -0.00488281 -0.00097656  0.00170898]
 [ 0.0065918   0.0090332  -0.00537109 -0.00390625  0.00073242  0.00317383
   0.00537109 -0.00561523 -0.00292969  0.00073242]
 [ 0.01025391  0.0078125  -0.00683594 -0.0012207   0.00097656  0.00488281
   0.00585938 -0.00512695 -0.00512695  0.00219727]
 [ 0.01000977  0.00683594 -0.00585938 -0.00463867  0.00024414  0.00415039
   0.00683594 -0.00634766 -0.00317383  0.00048828]
 [ 0.00927734  0.00732422 -0.0065918  -0.00219727  0.0012207   0.00366211
   0.00683594 -0.00585938 -0.00317383 -0.00244141]
 [ 0.00634766  0.00732422 -0.00585938  0.          0.00073242  0.00219727
   0.00634766 -0.00683594 -0.00195312 -0.00024414]
 [ 0.00439453  0.00585938 -0.00439453 -0.00219727  0.00048828  0.00341797
   0.00683594 -0.00732422 -0.00170898  0.0012207 ]
 

In [ ]:
# Possible issues:
    #print(oneA((0.5857864376269051, 0.9508722706208323)))
    #print(oneB((0.5857864376269051, 0.9508722706208323)))
    # n=2, m=2, g=(0,1), x=(0,0), when z=(-2,-2), we get this point.
    # It is in the circle but not the square.
    # sum2 for (0,0) should be equal to 0 (?), but this point is messing it up.
    
    
    # Another possible issue:
    # The middle summation does not appear to be symmetric?
    # But maybe the outermost summation is symmetric... actually it doesn't seem like it?
    
    # print(sum2(2,g,x)) returns 0 for x=(0.5,0.5), which makes sense intuitively.
    # but sum3 for n=0 through 5, with x=(0.5,0.5) doesn't return 0 ??
    
    
# Testing code:
    #F = make_set_z(2,0,g,(0,0), show = True)
    #print(sum3(2,0,g,x))
    #print(sum3(2,1,g,x))
    #print(sum3(2,2,g,x, show=True))
    #print(sum3(2,3,g,x))
    #print(sum2(2,g,x))
    #print(phi(x,g))
    #graph(g)
    
    """
    x = np.linspace(0, 1, 5)
    y = np.linspace(0, 1, 4)

    X, Y = np.meshgrid(x, y)
    Z = f(X, Y)
    
    print(X)
    print(Y)
    print(Z)
    
    X, Y = np.meshgrid(x, y)
    
    Z = np.empty([len(y),len(x)])
    for i in range(len(x)):
        for j in range(len(y)):
            Z[j][i] = phi((x[i].item(),y[j].item()),g)
    
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot_wireframe(X, Y, Z, color='black')
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("phi")
    plt.show(
    
    """
    
    #plt.pcolor(X, Y, Z)
    #plt.imshow(Z,origin='lower',interpolation='bilinear')
    
    